In [6]:
import random

def generate_individual(chromosome_length):
    # Fungsi ini menghasilkan satu individu acak dengan panjang kromosom tertentu.
    return [random.randint(0, 100) for _ in range(chromosome_length)]

def initialize_population(population_size, chromosome_length):
    # Fungsi ini menghasilkan populasi awal dengan sejumlah individu.
    return [generate_individual(chromosome_length) for _ in range(population_size)]

# Contoh penggunaan
population_size = 10
chromosome_length = 5

population = initialize_population(population_size, chromosome_length)
print("Populasi Awal:")
for individual in population:
    print(individual)


Populasi Awal:
[90, 41, 39, 81, 20]
[12, 60, 96, 89, 63]
[81, 21, 78, 24, 92]
[50, 43, 14, 3, 66]
[25, 30, 40, 69, 25]
[31, 36, 23, 16, 69]
[14, 14, 41, 92, 82]
[57, 60, 94, 13, 82]
[29, 79, 12, 6, 72]
[0, 74, 84, 69, 44]


In [15]:
population[9][2]

84